<a href="https://colab.research.google.com/github/giakomorssi/Machine_Learning/blob/main/03_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering

*   Identify the proper number of clusters, and evaluate different options.
*   Describe the properties of the clusters you have identified.
*   Describe the properties of the customers belonging to each cluster

In [ ]:
# Cluster the data
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

silhouette_scores = []

X = df.drop(['customer_id', 'purchase_date', 'approved_date',	'handled_by_logistic_date',	'delivery_date',	'estimated_delivery_date', 'shipping_limit_date'], axis=1)

# Calculate within-cluster sum of squares for different values of k
wcss = []
for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    labels = kmeans.predict(X)
    silhouette_avg = silhouette_score(X, labels)
    silhouette_scores.append(silhouette_avg)

# Plot the elbow graph
plt.plot(range(2, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Plot the silhouette scores for each value of k
plt.plot(range(2, 11), silhouette_scores)
plt.title('Silhouette Method')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.show()

# Find the optimal value for k

In [ ]:
diffs = np.diff(wcss)
acceleration = np.diff(diffs)
optimal_k = acceleration.argmax() + 2
print("Optimal k value:", optimal_k)

In [ ]:
kmeans = KMeans(n_clusters=optimal_k, init='k-means++', max_iter=300, n_init=10)
kmeans.fit(X)

df['Cluster'] = kmeans.labels_
df['Cluster'].value_counts()

# Most Important Features

In [ ]:
# get cluster centers and feature names
cluster_centers = kmeans.cluster_centers_
feature_names = df.columns

# loop through each cluster center and print top 3 most important features
for i, center in enumerate(cluster_centers):
    top_features = sorted(zip(feature_names, center), key=lambda x: x[1], reverse=True)[:3]
    print(f"Top 3 features for cluster {i+1}: {top_features}")